In [1]:
# importing the required packages
import pandas as pd
import numpy as np
import sqlite3
mydb = sqlite3.connect('coppermodel.db')
mycursor = mydb.cursor()

In [6]:
# retrive the datas from sql saved datas
df=pd.read_sql_query('select * from  copper',mydb)
df

,quantity_tons,country,status,item_type,thickness,width,selling_price,days_between
0,54.151139,28.0,2,5,2.00,1500.000,854.0,91.0
1,54.151139,28.0,2,5,2.00,1500.000,854.0,91.0
2,151.455482,25.0,2,5,0.80,1210.000,1047.0,0.0
3,151.455482,25.0,2,5,1.50,1265.000,983.0,0.0
4,3.522613,30.0,0,5,0.38,1125.000,243.0,91.0
...,...,...,...,...,...,...,...,...
165523,32.216122,25.0,1,4,6.00,1980.375,552.0,30.0
165524,66.431473,30.0,1,4,3.85,1525.000,458.0,30.0
165525,63.859521,30.0,1,4,5.85,1525.000,457.0,30.0
165526,56.028562,30.0,1,4,6.45,1500.000,458.0,30.0


In [25]:
# spliting the datas
from sklearn.model_selection import train_test_split

# Select features by including the first two columns and all columns from the fourth onwards
x = df.loc[:, list(df.columns)[:2] + list(df.columns)[3:]]

# Select the target column
y = df.loc[:, ["status"]]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=7)


In [14]:
# to check wether the data is balanced or not
df["status"].value_counts()/len(df)*100

status
1    75.475448
0    24.524552
Name: count, dtype: float64

In [ ]:
## we can do lable encoding for this status coloumn
#df["status"]=df["status"].map({"Wonderful":8,"Offered":7,"Offerable":6,"To be approved":5,"Revised":4,"Not lost for AM":3,"Won":2,"Lost":1,"Draft":0})

In [10]:
# maping the status column as 0 and 1 by its nature
df["status"]=df["status"].map({8:1,7:1,6:1,5:1,4:0,3:1,2:1,1:0,0:0})

In [26]:
# the data is imbalanced so by 
from imblearn.combine import SMOTEENN
cc=SMOTEENN()
x_trainc,y_trainc=cc.fit_resample(x_train,y_train)

In [32]:
# now it is balanced 
y_trainc.value_counts()/len(y_trainc)*100

status
0         55.376988
1         44.623012
Name: count, dtype: float64

In [2]:
# fiting and ckecking which depth is sutiable in decisiontree clasifier and using the cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
for i in range (1,25):
    model=DecisionTreeClassifier(max_depth=i,random_state=333)
    model.fit(x_trainc,y_trainc)
    cross_va=np.mean(cross_val_score(model,x_train,y_train,cv=10))
    print(f"the r2 score of the decisiontreethe depth is {i},score{model.score(x_test,y_test)},cross validation score {cross_va}")

KeyboardInterrupt: 

In [68]:
# by using the cross validation score the depthe 23 is the best hyperparameter for this model decisiontreeclasifier

from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(max_depth=23,random_state=333)
model.fit(x_trainc,y_trainc)
model.score(x_test,y_test)
from sklearn.metrics import f1_score
print(f1_score(y_test,model.predict(x_test)))


0.8900558584473358


In [54]:
# by using the random forest checking the best model 
#from sklearn.ensemble import RandomForestClassifier
#import numpy as np
#from sklearn.model_selection import cross_val_score
#model = RandomForestClassifier( max_depth=3, max_features='sqrt', random_state=42)
# Assuming x_train and y_train are already defined

#for n in [10,50,100,150,200,250,300,350,400,500,600]:
    model = RandomForestClassifier(n_estimators=n, max_depth=3, max_features='sqrt', random_state=42)
    cross_val_scores = np.mean(cross_val_score(model, x_trainc, y_trainc, cv=5))
    print(f"n_estimators={n}, crossvalidation score{cross_val_scores}")

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

n_estimators=10, crossvalidation score0.756335803386975


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

n_estimators=50, crossvalidation score0.7664980803817982


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

...

n_estimators=400, crossvalidation score0.7696059805029055


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

n_estimators=500, crossvalidation score0.7694037599629101


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConvers

n_estimators=600, crossvalidation score0.7699205042241056


In [99]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import cross_val_score

model=RandomForestClassifier()
model.fit(x_trainc,y_trainc)
model.score(x_test,y_test)
print(f1_score(model.predict(x_test),y_test))

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9052075673507587


also we checking the xgboost model to check the r2 score to find which model is best 

In [81]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming x_train, y_train are already defined

for lam in [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
    model = XGBClassifier(learning_rate=lam)
    cross_val_scores = np.mean(cross_val_score(model, x_trainc, y_trainc, cv=10))
    print(f"lambda={lam}, crossvalidation score={cross_val_scores}")

lambda=0.1, crossvalidation score=0.886656861375091
lambda=0.2, crossvalidation score=0.9122912883143911
lambda=0.3, crossvalidation score=0.9246853004668196
lambda=0.4, crossvalidation score=0.9315226305988243
lambda=0.5, crossvalidation score=0.9358886592553054
lambda=0.6, crossvalidation score=0.9398726531279348
lambda=0.7, crossvalidation score=0.9402096661430679
lambda=0.8, crossvalidation score=0.9415876179385443
lambda=0.9, crossvalidation score=0.9420669367782116
lambda=1, crossvalidation score=0.9425012798349484


also we checking the xgboost model to check the r2 score to find which model is best 

In [85]:
model=XGBClassifier(learning_rate=1)
model.fit(x_trainc,y_trainc)
print(model.score(x_test,y_test))
print(f1_score(y_test,model.predict(x_test)))



0.8366197863805519
0.8869946012803156


also we checking the logistic regresion model to check the r2 score to find which model is best 

In [98]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_trainc,y_trainc)
print(f1_score(model.predict(x_test),y_test))

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6607552846172117


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


there fore the best model in the clasifier is randomforest